In [2]:
from cmapPy.pandasGEXpress.parse import parse
import pandas as pd

In [6]:
data = parse("C:/Users/ngomi/Downloads/Documents/level5_beta_trt_oe_n34171x12328.gctx").data_df
gene_info = pd.read_csv("../data/geneinfo_beta.txt", delimiter="\t")
gene_dict = {}
for i in range(len(gene_info.index)):
    gene_dict[str(gene_info["gene_id"][i])] = gene_info["gene_symbol"][i]

In [12]:
up_gene_file = "../data/Example up genes.txt"
down_gene_file = "../data/Example down genes.txt"

def read_genes(file_name):
    with open(file_name) as file:
        return [gene.strip() for gene in file.readlines()]

up_genes = read_genes(up_gene_file)
down_genes = read_genes(down_gene_file)

In [22]:
n_expression = len(data.columns)
n_ref_gene = len(data.index)

def calc_es_score(ref_df, gene_list):
    cumsum_score = 0
    es_score = 0
    n = len(ref_df.index)
    ns = len(gene_list)
    nr = 0
    for i in range(n):
        if gene_dict[ref_df.index[i]] in gene_list:
            nr += abs(ref_df[i])
    for i in range(n):
        if gene_dict[ref_df.index[i]] in gene_list:
            cumsum_score += abs(ref_df[i])/nr
        else:
            cumsum_score -= 1/(n-ns)

        if abs(cumsum_score) > abs(es_score): es_score = cumsum_score
    return es_score
def cmap(up_gene_list, down_gene_list):
    up_gene_list = set(up_gene_list)
    down_gene_list = set(down_gene_list)
    c_scores = []
    for cid in data.columns:
        df = data[cid]
        df = df.sort_values(ascending=False)
        up_es_score = calc_es_score(df, up_gene_list)
        down_es_score = calc_es_score(df, down_gene_list)
        if up_es_score*down_es_score < 0:
            c_scores.append((up_es_score - down_es_score)/2)
        else:
            c_scores.append(0)
    return pd.DataFrame({"expression": data.columns, "c_score": c_scores})

In [23]:
cmap(up_genes, down_genes)

C:\Users\ngomi\AppData\Local\Temp\ipykernel_15460\2498749597.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  nr += abs(ref_df[i])
C:\Users\ngomi\AppData\Local\Temp\ipykernel_15460\2498749597.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cumsum_score += abs(ref_df[i])/nr


0.3132934410015845 -0.33898893164488314
0.15345214244347416 0.12537390076920105
0.2959959241843845 0.18441385711896854
0.0931152509360169 0.20828515086741206
0.2171626322663239 -0.20674489276461053
0.2856296093563266 0.17092900693054242
0.21078438926905427 0.20317987736207446
0.11567691255029568 0.24034312249222772
0.16416544971038324 0.20079637429302094
0.409982935569682 0.20438098509618086
0.30816639090661446 0.35104115331249425
0.2219181786254092 0.22845974011094553
-0.21235470321773145 0.16918936903481396
-0.1294797277550324 -0.20612610744916365
0.15924651175582652 -0.2293116771928959
-0.24966886715586764 0.26268276844322763
-0.22209609381899725 -0.14330006542024532
-0.17727232329018486 -0.16916784142939043
-0.2711699091311291 -0.16882122999436042
-0.42041307060069183 0.3309157449839473
-0.3558899110176151 -0.21175855581429148
0.1483206349359311 -0.1897277116812002
-0.36962983877023864 -0.1974333661085562
-0.4081734997294953 0.297798474889369
-0.28739721537055607 0.1883474997597037

KeyboardInterrupt: 